In [1]:
import pandas as pd
import numpy as np

#

In [2]:
# 4) Use your function to obtain a connection to the employees database

In [3]:
from env import get_db_url #function in env file, also, literally never print url in notebook unless you're debugging, even then make sure to erase before you push 

url = get_db_url('employees')

#from review
sql = """
SELECT * FROM employees
"""
employees = pd.read_sql(sql, url)
employees.shape

(300024, 6)

In [4]:
# from review
sql = """
SELECT * FROM titles
"""
titles = pd.read_sql(sql, url)
titles.shape

(443308, 4)

In [5]:
#query = '''
#SELECT * FROM employees LIMIT 5'''

#pd.read_sql(query, url)

# 

In [6]:
# 5) Once you have successfully run a query:
    # a)Intentionally make a typo in the database url. What kind of error message do you see?
      #Operational Error
#url = f'mysql+pymysql://{user}:{password}@{host}/employeess'

#query = '''
#SELECT * FROM employees LIMIT 5'''

#pd.read_sql(query, url)
    # b) Intentionally make an error in your SQL query. What does the error message look like?
    #ProgrammingError
#query = '''
#SLECT * FROM employees LIMIT 5'''

#pd.read_sql(query, url)    

In [7]:
# 6) Read the employees and titles tables into two separate DataFrames.

In [8]:
#query2 = """
#SELECT * FROM employees"""
#pd.read_sql(query2, url)  #300024 rows × 6 columns

In [9]:
#query3 = '''
#SELECT * FROM titles'''
#pd.read_sql(query3, url) # 443308 rows × 4 columns

In [10]:
# 7) How many rows and columns do you have in each DataFrame? 
# Is that what you expected? --- I did not expect so many entries.
# employees: 300024 rows × 6 columns
# titles: 443308 rows × 4 columns

In [11]:
# 8) Display the summary statistics for each DataFrame.

In [12]:
#query2 = """
#SELECT * FROM employees"""
#pd.read_sql(query2, url).describe() 

In [13]:
#query3 = '''
#SELECT * FROM titles'''
#pd.read_sql(query3, url).describe() 

In [14]:
# 9) How many unique titles are in the titles DataFrame?

pd.read_sql("SELECT COUNT(DISTINCT title) AS distinct_title_count FROM titles", url)

,distinct_title_count
0,7


In [15]:
#from review, Pandas style
len(titles.title.unique()) 

7

In [16]:
# 10) What is the oldest date in the to_date column? 
pd.read_sql('SELECT MIN(to_date) FROM titles', url) #1985-03-01

,MIN(to_date)
0,1985-03-01


In [17]:
#Pandas style
titles.to_date.min()

datetime.date(1985, 3, 1)

In [18]:
# 11) What is the most recent date in the to_date column?
pd.read_sql('SELECT MAX(to_date) FROM titles', url) #9999-01-01

#pandas style doesn't work here bc of the very forward into the future to_date used for current employees

,MAX(to_date)
0,9999-01-01


---

In [19]:
# Exercises II
# Copy the users and roles DataFrames from the examples above. 

# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users



,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [20]:

# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [21]:
#What is the result of using a right join on the DataFrames? 
right_join = pd.merge(users, roles, left_on = 'role_id', right_on = 'id', how = 'right', indicator = True)
right_join

# have 3 columns w/ NaN 

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


In [22]:
# 3) What is the result of using an outer join on the DataFrames?

pd.merge(users,
        roles, 
        left_on = 'role_id',
        right_on = 'id',
        how = 'outer',
        indicator = True)



,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [23]:
pd.merge(users,
        roles, 
        left_on = 'role_id',
        right_on = 'id',
        how = 'outer').drop(columns = 'role_id').rename(columns = {'id_x':'id',
                                                       'name_x':'employee',
                                                       'id_y': 'role_id',
                                                       'name_y': 'role'}
                                                       )


,id,employee,role_id,role
0,1.0,bob,1.0,admin
1,2.0,joe,2.0,author
2,3.0,sally,3.0,reviewer
3,4.0,adam,3.0,reviewer
4,5.0,jane,NaN,NaN
5,6.0,mike,NaN,NaN
6,NaN,NaN,4.0,commenter


In [24]:
# 4) What happens if you drop the foreign keys from the DataFrames and try to merge them?

users



,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [25]:
#drop foreign key
u_drop_key = users.drop(columns = 'role_id')
u_drop_key

# with drop in role_id, joins may be confusing

,id,name
0,1,bob
1,2,joe
2,3,sally
3,4,adam
4,5,jane
5,6,mike


In [26]:
# 5) Load the mpg dataset from PyDataset. 
from pydataset import data
data('mpg', show_doc = True)
mpg = data('mpg')

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [27]:
# 6) Output and read the documentation for the mpg dataset.
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


In [28]:
# 7) How many rows and columns are in the dataset? 
# 234 x 11

In [29]:
# 8) Check out your column names and perform any cleanup you may want on them.
mpg.columns.tolist()

['manufacturer',
 'model',
 'displ',
 'year',
 'cyl',
 'trans',
 'drv',
 'cty',
 'hwy',
 'fl',
 'class']

In [30]:
mpg.columns = ['manufacturer', 'model', 'displacement', 'year', 'cylinder', 'transmission', 'drive', 'city', 'highway', 'fuel', 'class']

In [31]:
# 9) Display the summary statistics for the dataset.
mpg.describe()

,displacement,year,cylinder,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [32]:
# 10) How many different manufacturers are there?
mpg.manufacturer.describe() #15

count       234
unique       15
top       dodge
freq         37
Name: manufacturer, dtype: object

In [33]:
# 11) How many different models are there?
mpg.model.describe()

count             234
unique             38
top       caravan 2wd
freq               11
Name: model, dtype: object

In [34]:
mpg.model.nunique()

38

In [35]:
# 12) Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain 
# the difference between highway and city mileage for each car.
mpg = mpg.assign(mileage_difference = mpg.highway - mpg.city)

In [36]:
# 13) Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.
mpg = mpg.assign(average_mileage = (mpg.highway + mpg.city) / 2)

In [37]:
# 14) Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

#mpg.transmission.describe()
mpg['is_automatic'] = mpg.transmission.str.startswith('a')
mpg

,manufacturer,model,displacement,year,cylinder,transmission,drive,city,highway,fuel,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


In [38]:
# 15) Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?
mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending = False).round(2)

#Honda

manufacturer
honda         28.50
volkswagen    25.07
hyundai       22.75
subaru        22.43
audi          22.03
toyota        21.72
pontiac       21.70
nissan        21.35
chevrolet     18.45
ford          16.68
mercury       15.62
jeep          15.56
dodge         15.54
lincoln       14.17
land rover    14.00
Name: average_mileage, dtype: float64

In [39]:
# 16) Do automatic or manual cars have better miles per gallon?

#create new column that outputs if the car is automatic or manual
mpg['auto_or_manual'] = np.where(mpg.transmission.str.startswith('a'), 'auto', 'manual')
mpg

,manufacturer,model,displacement,year,cylinder,transmission,drive,city,highway,fuel,class,mileage_difference,average_mileage,is_automatic,auto_or_manual
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True,auto
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False,manual
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False,manual
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True,auto
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True,auto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True,auto
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False,manual
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True,auto
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False,manual


In [40]:
# groupby auto_or_manual, subset by highway and city and get mean
mpg.groupby('auto_or_manual')[['highway' , 'city']].mean() #manual is better for both city and highway

,highway,city
auto_or_manual,,
auto,22.292994,15.968153
manual,25.779221,18.675325


In [52]:
# Exercises III
# Use your get_db_url function to help you explore the data from the chipotle database. 

from env import get_db_url

url2 = get_db_url('chipotle') #named bc I don't want to overwrite previous url/sql variable
sql2 = '''                     
SELECT * FROM orders
'''
chipotle = pd.read_sql(sql2, url2) 
chipotle.shape

# can also use: pd.read_sql('SELECT * FROM <TABLE NAME> LIMIT 5', get_db_url ('<DB NAME>'))
## pd.read_sql('SELECT * FROM orders', get_db_url('chipotle'))

(4622, 6)

In [42]:
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


In [43]:
chipotle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  4622 non-null   int64 
 1   order_id            4622 non-null   int64 
 2   quantity            4622 non-null   int64 
 3   item_name           4622 non-null   object
 4   choice_description  4622 non-null   object
 5   item_price          4622 non-null   object
dtypes: int64(3), object(3)
memory usage: 216.8+ KB


In [61]:
# 2) What is the total price for each order?

chipotle['item_price'] = chipotle['item_price'].astype(float) #convert to float
#chipotle.head()

chipotle_totals = chipotle.groupby('order_id').item_price.sum() #groupby order_id

chipotle_totals.head()

order_id
1    11.56
2    16.98
3    12.67
4    21.00
5    13.70
Name: item_price, dtype: float64

In [63]:
# 3) What are the most popular 3 items?

popular_items = chipotle.groupby('item_name').quantity.sum().sort_values(ascending = False).head(3)
popular_items

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

In [67]:
# 4) Which item has produced the most revenue?
money_maker = chipotle.groupby('item_name').item_price.sum().nlargest(1, keep = 'all')
money_maker

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

In [68]:
money_loser = chipotle.groupby('item_name').item_price.sum().nsmallest(1, keep = 'all')
money_loser

item_name
Chips and Mild Fresh Tomato Salsa    3.0
Name: item_price, dtype: float64

In [47]:
# 5) Join the employees and titles DataFrames together.

In [48]:
# 6) For each title, find the hire date of the employee that was hired most recently 
# with that title.

In [49]:
# 7) Write the code necessary to create a cross tabulation of the number of titles by 
# department. (Hint: this will involve a combination of SQL code to pull the necessary 
# data and python/pandas code to perform the manipulations.)